In [366]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch
import sys
warnings.simplefilter('ignore')
from sklearn.model_selection import GridSearchCV
from catboost import CatBoostRegressor
import re
from pymorphy2 import MorphAnalyzer
from nltk.corpus import stopwords
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import r2_score
from catboost import CatBoostRegressor
from sklearn.metrics import mean_absolute_error

In [367]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth',100)
np.set_printoptions(threshold=sys.maxsize)

In [368]:
df = pd.concat([pd.read_csv(r"C:\Users\nicka\cian_parsing_result_sale_1_7_vidnyj_18_Jun_2023_23_28_58_908950.csv",sep = ';',encoding = 'cp1251'),
                     pd.read_csv(r"C:\Users\nicka\cian_parsing_result_sale_7_20_vidnyj_18_Jun_2023_23_12_06_369578.csv",sep = ';',encoding = 'cp1251')])
df.sample(10)

,author,link,city,deal_type,accommodation_type,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters,phone,district,street,underground,residential_complex
148,ФСК,https://vidnoye.cian.ru/sale/flat/260871983/,Видный,sale,flat,1,16,-1,202.0,11106600,2023,30.0,8.0,74951346390,NaN,NaN,NaN,NaN
123,ID 95922805,https://vidnoye.cian.ru/sale/flat/286527135/,Видный,sale,flat,1,4,2,60.7,11579926,-1,33.0,12.0,79645679593,NaN,NaN,NaN,NaN
139,ID 29097,https://vidnoye.cian.ru/sale/flat/272480349/,Видный,sale,flat,10,27,3,78.0,11300000,2014,45.0,10.0,79060427747,NaN,NaN,NaN,NaN
36,ПИК,https://vidnoye.cian.ru/sale/flat/278399401/,Видный,sale,flat,2,25,2,39.3,6741884,2024,23.0,5.0,74951389937,NaN,NaN,NaN,NaN
76,ДСК-1,https://vidnoye.cian.ru/sale/flat/281306172/,Видный,sale,flat,2,21,-1,202.0,10864440,2025,28.0,12.0,74951346097,NaN,NaN,NaN,NaN
16,ФСК,https://vidnoye.cian.ru/sale/flat/273655928/,Видный,sale,flat,9,10,-1,202.0,8997750,2024,27.0,7.0,74951346390,NaN,NaN,NaN,NaN
72,ФСК,https://vidnoye.cian.ru/sale/flat/269178679/,Видный,sale,flat,1,10,-1,202.0,9918174,2024,31.0,11.0,74951346390,NaN,NaN,NaN,NaN
92,MR Group,https://vidnoye.cian.ru/sale/flat/284019039/,Видный,sale,flat,15,17,-1,202.0,11246915,2024,-1.0,-1.0,74951346117,NaN,NaN,NaN,NaN
82,Тройка РЭД,https://vidnoye.cian.ru/sale/flat/288175532/,Видный,sale,flat,11,25,3,69.6,11052480,2026,37.0,-1.0,74951342841,NaN,NaN,NaN,NaN
89,ПЕРСПЕКТИВА24-ЮГО-ЗАПАД,https://vidnoye.cian.ru/sale/flat/288646821/,Видный,sale,flat,4,14,2,51.0,7750000,2023,30.0,6.0,79663573381,NaN,NaN,NaN,NaN


In [386]:
df =  pd.read_csv(r"C:\Users\nicka\cian_parsing_result_sale_1_7_dolgoprudnyj_18_Jun_2023_23_47_25_298144.csv",sep = ';',encoding = 'cp1251')

In [387]:
df

,author,link,city,deal_type,accommodation_type,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters,phone,district,street,underground,residential_complex
0,NaN,https://dolgoprudny.cian.ru/sale/flat/267244381/,Долгопрудный,sale,flat,3,23,2,44.9,8403910,2023,-1.0,-1.0,74951384886,NaN,NaN,Водники,NaN
1,Ирина Василевская,https://dolgoprudny.cian.ru/sale/flat/286646798/,Долгопрудный,sale,flat,12,17,3,78.0,12500000,2015,46.0,13.0,79660635242,NaN,NaN,Хлебниково,NaN
2,NaN,https://dolgoprudny.cian.ru/sale/flat/267244369/,Долгопрудный,sale,flat,2,23,2,41.4,8756483,2023,-1.0,-1.0,74951384886,NaN,NaN,Водники,NaN
3,Homeapp,https://dolgoprudny.cian.ru/sale/flat/287428495/,Долгопрудный,sale,flat,9,23,2,60.0,10500000,2022,37.0,10.0,79660531406,NaN,NaN,Долгопрудная,NaN
4,NaN,https://dolgoprudny.cian.ru/sale/flat/263572070/,Долгопрудный,sale,flat,2,23,2,50.7,10302945,2023,-1.0,-1.0,74951384886,NaN,NaN,Водники,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Александр Кухновец,https://dolgoprudny.cian.ru/sale/flat/287362209/,Долгопрудный,sale,flat,3,24,2,63.7,13158500,2020,31.0,17.0,79670285494,NaN,NaN,Долгопрудная,NaN
192,Сергей Желтаков,https://dolgoprudny.cian.ru/sale/flat/288668593/,Долгопрудный,sale,flat,3,5,2,39.0,8900000,1960,28.0,5.0,79651395933,NaN,NaN,Долгопрудная,NaN
193,РАУИ,https://dolgoprudny.cian.ru/sale/flat/250995393/,Долгопрудный,sale,flat,18,23,2,55.0,10300000,2021,20.0,11.0,79636291183,NaN,NaN,Водники,NaN
194,Владис,https://dolgoprudny.cian.ru/sale/flat/288419215/,Долгопрудный,sale,flat,1,18,2,55.1,10990000,2015,27.0,9.0,79645672951,NaN,NaN,Ховрино,NaN


In [388]:
df_cop = df.copy()

## Дополнение living_meters и kitchen_meters

In [389]:
df_cop

,author,link,city,deal_type,accommodation_type,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters,phone,district,street,underground,residential_complex
0,NaN,https://dolgoprudny.cian.ru/sale/flat/267244381/,Долгопрудный,sale,flat,3,23,2,44.9,8403910,2023,-1.0,-1.0,74951384886,NaN,NaN,Водники,NaN
1,Ирина Василевская,https://dolgoprudny.cian.ru/sale/flat/286646798/,Долгопрудный,sale,flat,12,17,3,78.0,12500000,2015,46.0,13.0,79660635242,NaN,NaN,Хлебниково,NaN
2,NaN,https://dolgoprudny.cian.ru/sale/flat/267244369/,Долгопрудный,sale,flat,2,23,2,41.4,8756483,2023,-1.0,-1.0,74951384886,NaN,NaN,Водники,NaN
3,Homeapp,https://dolgoprudny.cian.ru/sale/flat/287428495/,Долгопрудный,sale,flat,9,23,2,60.0,10500000,2022,37.0,10.0,79660531406,NaN,NaN,Долгопрудная,NaN
4,NaN,https://dolgoprudny.cian.ru/sale/flat/263572070/,Долгопрудный,sale,flat,2,23,2,50.7,10302945,2023,-1.0,-1.0,74951384886,NaN,NaN,Водники,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
191,Александр Кухновец,https://dolgoprudny.cian.ru/sale/flat/287362209/,Долгопрудный,sale,flat,3,24,2,63.7,13158500,2020,31.0,17.0,79670285494,NaN,NaN,Долгопрудная,NaN
192,Сергей Желтаков,https://dolgoprudny.cian.ru/sale/flat/288668593/,Долгопрудный,sale,flat,3,5,2,39.0,8900000,1960,28.0,5.0,79651395933,NaN,NaN,Долгопрудная,NaN
193,РАУИ,https://dolgoprudny.cian.ru/sale/flat/250995393/,Долгопрудный,sale,flat,18,23,2,55.0,10300000,2021,20.0,11.0,79636291183,NaN,NaN,Водники,NaN
194,Владис,https://dolgoprudny.cian.ru/sale/flat/288419215/,Долгопрудный,sale,flat,1,18,2,55.1,10990000,2015,27.0,9.0,79645672951,NaN,NaN,Ховрино,NaN


In [390]:
df_cop.drop(columns = 'author',inplace = True)
df_cop.drop(columns = 'link',inplace = True)
df_cop['city'] = "dolgoprudny"
df_cop.drop(columns = 'deal_type',inplace = True)
df_cop.drop(columns = 'accommodation_type',inplace = True)
df_cop.drop(columns = 'street',inplace = True)
df_cop.drop(columns = 'residential_complex',inplace = True)
df_cop.drop(columns = 'phone',inplace = True)

In [391]:
df_cop

,city,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters,district,underground
0,dolgoprudny,3,23,2,44.9,8403910,2023,-1.0,-1.0,NaN,Водники
1,dolgoprudny,12,17,3,78.0,12500000,2015,46.0,13.0,NaN,Хлебниково
2,dolgoprudny,2,23,2,41.4,8756483,2023,-1.0,-1.0,NaN,Водники
3,dolgoprudny,9,23,2,60.0,10500000,2022,37.0,10.0,NaN,Долгопрудная
4,dolgoprudny,2,23,2,50.7,10302945,2023,-1.0,-1.0,NaN,Водники
...,...,...,...,...,...,...,...,...,...,...,...
191,dolgoprudny,3,24,2,63.7,13158500,2020,31.0,17.0,NaN,Долгопрудная
192,dolgoprudny,3,5,2,39.0,8900000,1960,28.0,5.0,NaN,Долгопрудная
193,dolgoprudny,18,23,2,55.0,10300000,2021,20.0,11.0,NaN,Водники
194,dolgoprudny,1,18,2,55.1,10990000,2015,27.0,9.0,NaN,Ховрино


In [392]:
for col in df_cop.drop(columns = ['city','district','underground']).columns:
    df_cop = df_cop.drop(df_cop[df_cop[col] < 0].index)
df_cop

,city,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters,district,underground
1,dolgoprudny,12,17,3,78.0,12500000,2015,46.0,13.0,NaN,Хлебниково
3,dolgoprudny,9,23,2,60.0,10500000,2022,37.0,10.0,NaN,Долгопрудная
5,dolgoprudny,11,18,2,56.0,10100000,2017,25.0,10.0,NaN,Водники
11,dolgoprudny,2,4,2,42.4,7500000,1962,23.0,6.0,NaN,Долгопрудная
13,dolgoprudny,1,12,3,72.9,12500000,2002,44.0,12.0,NaN,Водники
...,...,...,...,...,...,...,...,...,...,...,...
191,dolgoprudny,3,24,2,63.7,13158500,2020,31.0,17.0,NaN,Долгопрудная
192,dolgoprudny,3,5,2,39.0,8900000,1960,28.0,5.0,NaN,Долгопрудная
193,dolgoprudny,18,23,2,55.0,10300000,2021,20.0,11.0,NaN,Водники
194,dolgoprudny,1,18,2,55.1,10990000,2015,27.0,9.0,NaN,Ховрино


In [393]:
df_cop.district.fillna('unk',inplace = True)
df_cop.underground.fillna('unk',inplace = True)

In [394]:
df_cop.dropna(inplace = True)
df_cop = df_cop.drop_duplicates()
df_cop

,city,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters,district,underground
1,dolgoprudny,12,17,3,78.0,12500000,2015,46.0,13.0,unk,Хлебниково
3,dolgoprudny,9,23,2,60.0,10500000,2022,37.0,10.0,unk,Долгопрудная
5,dolgoprudny,11,18,2,56.0,10100000,2017,25.0,10.0,unk,Водники
11,dolgoprudny,2,4,2,42.4,7500000,1962,23.0,6.0,unk,Долгопрудная
13,dolgoprudny,1,12,3,72.9,12500000,2002,44.0,12.0,unk,Водники
...,...,...,...,...,...,...,...,...,...,...,...
191,dolgoprudny,3,24,2,63.7,13158500,2020,31.0,17.0,unk,Долгопрудная
192,dolgoprudny,3,5,2,39.0,8900000,1960,28.0,5.0,unk,Долгопрудная
193,dolgoprudny,18,23,2,55.0,10300000,2021,20.0,11.0,unk,Водники
194,dolgoprudny,1,18,2,55.1,10990000,2015,27.0,9.0,unk,Ховрино


In [395]:
df_cop.isnull().sum()

city                    0
floor                   0
floors_count            0
rooms_count             0
total_meters            0
price                   0
year_of_construction    0
living_meters           0
kitchen_meters          0
district                0
underground             0
dtype: int64

In [397]:
df_cop = pd.concat([df_cop[['city','district','underground']],df_cop.drop(columns = ['city','district','underground'])],axis = 1)
df_cop

,city,district,underground,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters
1,dolgoprudny,unk,Хлебниково,12,17,3,78.0,12500000,2015,46.0,13.0
3,dolgoprudny,unk,Долгопрудная,9,23,2,60.0,10500000,2022,37.0,10.0
5,dolgoprudny,unk,Водники,11,18,2,56.0,10100000,2017,25.0,10.0
11,dolgoprudny,unk,Долгопрудная,2,4,2,42.4,7500000,1962,23.0,6.0
13,dolgoprudny,unk,Водники,1,12,3,72.9,12500000,2002,44.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...
191,dolgoprudny,unk,Долгопрудная,3,24,2,63.7,13158500,2020,31.0,17.0
192,dolgoprudny,unk,Долгопрудная,3,5,2,39.0,8900000,1960,28.0,5.0
193,dolgoprudny,unk,Водники,18,23,2,55.0,10300000,2021,20.0,11.0
194,dolgoprudny,unk,Ховрино,1,18,2,55.1,10990000,2015,27.0,9.0


In [398]:
df_cop.reset_index(inplace = True, drop = True)

In [399]:
def ung(place):
    if place.find(' ') > 0:
        return place[:(place.find(' ')+1)]
    return place
df_cop['underground'] = df_cop['underground'].apply(ung)

In [400]:
df_cop

,city,district,underground,floor,floors_count,rooms_count,total_meters,price,year_of_construction,living_meters,kitchen_meters
0,dolgoprudny,unk,Хлебниково,12,17,3,78.0,12500000,2015,46.0,13.0
1,dolgoprudny,unk,Долгопрудная,9,23,2,60.0,10500000,2022,37.0,10.0
2,dolgoprudny,unk,Водники,11,18,2,56.0,10100000,2017,25.0,10.0
3,dolgoprudny,unk,Долгопрудная,2,4,2,42.4,7500000,1962,23.0,6.0
4,dolgoprudny,unk,Водники,1,12,3,72.9,12500000,2002,44.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...
125,dolgoprudny,unk,Долгопрудная,3,24,2,63.7,13158500,2020,31.0,17.0
126,dolgoprudny,unk,Долгопрудная,3,5,2,39.0,8900000,1960,28.0,5.0
127,dolgoprudny,unk,Водники,18,23,2,55.0,10300000,2021,20.0,11.0
128,dolgoprudny,unk,Ховрино,1,18,2,55.1,10990000,2015,27.0,9.0


In [401]:
df_cop.insert(7,'price_per_m2',(df_cop['price']/df_cop['total_meters']).astype('int64'))

In [402]:
df_cop

,city,district,underground,floor,floors_count,rooms_count,total_meters,price_per_m2,price,year_of_construction,living_meters,kitchen_meters
0,dolgoprudny,unk,Хлебниково,12,17,3,78.0,160256,12500000,2015,46.0,13.0
1,dolgoprudny,unk,Долгопрудная,9,23,2,60.0,175000,10500000,2022,37.0,10.0
2,dolgoprudny,unk,Водники,11,18,2,56.0,180357,10100000,2017,25.0,10.0
3,dolgoprudny,unk,Долгопрудная,2,4,2,42.4,176886,7500000,1962,23.0,6.0
4,dolgoprudny,unk,Водники,1,12,3,72.9,171467,12500000,2002,44.0,12.0
...,...,...,...,...,...,...,...,...,...,...,...,...
125,dolgoprudny,unk,Долгопрудная,3,24,2,63.7,206569,13158500,2020,31.0,17.0
126,dolgoprudny,unk,Долгопрудная,3,5,2,39.0,228205,8900000,1960,28.0,5.0
127,dolgoprudny,unk,Водники,18,23,2,55.0,187272,10300000,2021,20.0,11.0
128,dolgoprudny,unk,Ховрино,1,18,2,55.1,199455,10990000,2015,27.0,9.0


In [403]:
df_cop.to_csv("dolgoprudny.csv")